23/06/2023 au 26/06/2023    
objectif : faire la comparaison avec et sans contexte des performances de SpaCyfishing  
contexte : reprise de 

# Evaluation comparative  du système d'entity linking de SpaCy

## code donné par CKP

In [14]:
texte = lire_fichier(path)
text=nlp(texte)  
for ent in text.ents:
        list_resultats.append([ent.text, ent.start_char, ent.end_char, ent.label_,texte[ent.start_char-200:ent.start_char],texte[ent.end_char:ent.end_char+200]])

## Première adaptation

In [3]:
import glob
import spacy
import json


def lire_fichier(chemin):
    with open (chemin, "r", encoding="utf-8") as fichier:
        texte=fichier.read()
        return texte

list_resultats=[]
path_corpora = "../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/*/*.txt"

for path in glob.glob(path_corpora):
    texte = lire_fichier(path)
    nlp = spacy.load("fr_core_news_sm")
    nlp.add_pipe("entityLinker", last=True)
    text=nlp(texte) 
    list_resultats=[]
    for ent in text.ents:
        list_resultats.append([ent.text, ent.start_char, ent.end_char, ent.label_,texte[ent.start_char-200:ent.start_char],ent.text,texte[ent.end_char:ent.end_char+200]])
print (list_resultats[0])
print (list_resultats[70])

['Norine Duclos', 76, 89, 'PER', '', " qu'ont\nlieu les plus egayantes veilles de notre vil-\nlage. Adonc, certain soir, comme j'entrais\nchez Norine, je la trouvai en train de precher\nses trois petiotes.\n- He MIandine, criait-elle, dpeche-t"]
['petiot\nRoux', 5007, 5018, 'PER', "t Jean-Claude;\nle dragon viendra pendant ce temps-la.\n- Je veux, au contraire, l'attendre pour\nchanter, dit la Rose d'un ton pique.\n- Chante, dit Gaspard, ou sinon...\n-- Tu deviens par trop bravache, ", ', dit Norine; tache un peu de te tenir\ntranquille.\n- Je vas chanter, moi, dit le vieux berger.l-\nMION VI1LAGE.\n- lEn ce cas-la, je me sauve, dit la petiote\nh Perpetue.\n- Non, dit le fieu de Toinon, so']


### Nous vérifions ensuite dans le texte si nous retrouvons bien les mêmes valeurs

"Depuis un nombre innombrable d’hivers, c’est dans la maison de Norine Duclos qu’ont lieu les plus égayantes veillées de notre village. Adonc, certain soir, comme j’entrais chez Norine, je la trouvai en train de prêcher ses trois petiotes. "

"- Chante, toi, la Rose, dit Jean-Claude;
le dragon viendra pendant ce temps-la.
- Je veux, au contraire, l'attendre pour
chanter, dit la Rose d'un ton pique.
- Chante, dit Gaspard, ou sinon...
-- Tu deviens par trop bravache, petiot
Roux, dit Norine; tache un peu de te tenir
tranquille.
- Je vas chanter, moi, dit le vieux berger.l-
MION VI1LAGE.
- lEn ce cas-la, je me sauve, dit la petiote
h Perpetue."

## Nous procédons à la NER puis appliquons l'entity linking aux entités hors contexte ...

En théorie, l'analyse du contexte est dans l'ADN même des systèmes d'entity linking. En effet, l'intérêt d'un tels outils est de pouvoir déterminer le sens d'un mot en fonction de son contexte. Cependant, le système d'entity liking que nous étudions ne semble pas vraiment tenir compte du contexte au vu de ses performances. Nous avons donc cherché à évaluer les performances du système en contexte et hors contexte. Pour accomplir cette expérimentation, nous avons extrait les entités nommées avec le système de REN de Spacy. Nous avons, d'autre part, extrait les entitées avec un contexte de quatre cent caractères. Nous avons ensuite soumis les deux à l'entity linker de SpaCy.
[demander à Caroline une référence vers son travail]

La REN de SpaCy présente le problème d'avoir elle-même des faiblesses et de proposer de nombreux faux positifs. "Adonc", "Ah !", "Jure-moi" sont identifiés comme des EN à cause de leur majuscule; certains mots anodins tel que "gens" ou "ble" sont également identifiés comme EN. L'entity linker ne parvient pas pour autant toujours à les lier. De même, les données bruitées sont reconnues REN à cause de leur forme inconnue comme "s0UII", "E.:c". L'entity linking semble dans la plupart des cas ne pas trouver de référent à associer. Il existe des données bruitées qui ont tout de même trouvé un référent dans la base de connaissance comme "FRUCTUBUx CBLLIER", l'association avec la constante physique "N_A" demeure cependant peu pertinente.
Peut-être un usage dérivé de cet outil pourrait être de répérer les faux positifs lors de la REN. Même si ce n'est pas le cas pour ce corpus, il y a toutefois des EN qui peuvent ne pas avoir de référent dans la base de connaissance car il peut s'agit de prénom exotique  ou de lieux fictifs non répertoriés sur wikidata.
 
Comme nous l'avons déjà démontré, la reconnaissance des entités pour l'Entity Linker fonctionne indépendemment de REN initialement implémentée dans SpaCy. Pour ce qui est reconnu commme une seule entité par la REN de SpaCy, l'Entity Linker peut y identifier plusieurs éléments. C'est essentiellement, le cas pour les entités composées de plusieurs mots comme "Rose, ma promise". Cela peut également être le cas pour un mot unique comme "cerez". Toutefois, une entité composée de plusieurs mots peut n'avoir qu'un seul référent dans la base de connaissance comme "Rose du déshonneur". 


In [ ]:
Il semble qu'en contexte le système soit moins sensible au bruit. 


In [4]:
import json
import spacy  # version 3.5
import glob
import matplotlib.pyplot as plt
import pandas as pd

def lire_fichier(chemin):
    with open(chemin, "r", encoding="utf-8") as fichier:
        texte = fichier.read()
        return texte

def stocker(chemin, contenu):
    with open(chemin, "w") as w:
        json.dump(contenu, w, indent=2)

modele = ["md"]
path_corpora = "../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/*/*.txt"
common_urls_per_file = []
common_urls = []
non_common_urls = []

#modele=["sm","md","lg"]
modele=["sm"]

for mod in modele:
    print("Pour le modèle:", mod, "\n")
    nlp = spacy.load("fr_core_news_" + mod)
    nlp.add_pipe("entityLinker", last=True)
    for path in glob.glob(path_corpora):
        print("Entrée:", path)
        texte = lire_fichier(path)
        doc = nlp(texte)
        list_resultats = []
        list_entites = []
        compteur = 0
        intersect = 0

        i = 1
        dico={}
        for ent in doc.ents:
            contexte = str(texte[ent.start_char-50:ent.start_char])+str(ent.text)+ str(texte[ent.end_char:ent.end_char+50])
            entite = ent.text
            
            print(entite)
            liste_url_entourage = []
            liste_url_entite = []
            doc2 = nlp(entite)
            doc3 = nlp(contexte)
            for ent in doc3._.linkedEntities:
                url_entourage = ent.get_url()
                liste_url_entourage.append(url_entourage)
            for ent in doc2._.linkedEntities:
                url_entite = ent.get_url()
                liste_url_entite.append(url_entite)
                for d in liste_url_entourage :
                    if d in liste_url_entite:
                        #print (d)
                        intersect = intersect + 1
            print (liste_url_entite)
            compteur +=1

Pour le modèle: sm 

Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/REF/ADAM_Mon-village_REF.txt
VEILLÉE
[]
Norine Duclos
['https://www.wikidata.org/wiki/Q21450529']
Adonc
[]
Norine
[]
Hé !
[]
Mandine
[]
dépêche-toi de balayer
['https://www.wikidata.org/wiki/Q180309', 'https://www.wikidata.org/wiki/Q7813319']
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Rose
['https://www.wikidata.org/wiki/Q519034']
Habile
[]
Désirée !
['https://www.wikidata.org/wiki/Q945470']
Faites excuse
[]
Cellier
['https://www.wikidata.org/wiki/Q21501975']
Norine
[]
Saint-Brunelle
[]
Armandine
[]
Rose
['https://www.wikidata.org/wiki/Q519034']
Désirée
['https://www.wikidata.org/wiki/Q945470']
réserve de la Rose
['https://www.wikidata.org/wiki/Q519034']
Norine
[]
Rose
['https://www.wikidata.org/wiki/Q519034']
garçonnière
['https://www.wikidata.org/wiki/Q3822506']
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
couseuses
[]
couseuses
[]
Roux
['https://www.wikidata.org/wiki/Q201157']
Taisez
[]
Pe

[]
Norine
[]
Rose
['https://www.wikidata.org/wiki/Q519034']
Pierre
['https://www.wikidata.org/wiki/Q40747']
La Rose
['https://www.wikidata.org/wiki/Q519034']
Maman
['https://www.wikidata.org/wiki/Q180887']
Tiens
[]
Rose
['https://www.wikidata.org/wiki/Q519034']
M'aimeras-tu
['https://www.wikidata.org/wiki/Q180309']
Rose
['https://www.wikidata.org/wiki/Q519034']
Oh !
[]
Pierre
['https://www.wikidata.org/wiki/Q40747']
Oh !
[]
Pierre
['https://www.wikidata.org/wiki/Q40747']
Ça
[]
Rose
['https://www.wikidata.org/wiki/Q519034']
Jure
[]
Pierre
['https://www.wikidata.org/wiki/Q40747']
dépêche-toi
['https://www.wikidata.org/wiki/Q180309', 'https://www.wikidata.org/wiki/Q7813319']
Norine
[]
Oh !
[]
Rose
['https://www.wikidata.org/wiki/Q519034']
Pierre
['https://www.wikidata.org/wiki/Q40747']
vois-tu
[]
Rose
['https://www.wikidata.org/wiki/Q519034']
La Rose
['https://www.wikidata.org/wiki/Q519034']
Pierre
['https://www.wikidata.org/wiki/Q40747']
Crois-tu
[]
Pierre
['https://www.wikidata.org/wiki

['https://www.wikidata.org/wiki/Q25269']
Perpétue
[]
Jeacquet
[]
Toinon
[]
Monsieur
['https://www.wikidata.org/wiki/Q1092777']
Clarisse Manot
['https://www.wikidata.org/wiki/Q733658', 'https://www.wikidata.org/wiki/Q2777738']
Saint-Brunelle
[]
Pâques
[]
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Hé !
[]
lurot
[]
lurot
[]
Va
[]
Roux
['https://www.wikidata.org/wiki/Q201157']
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Adonc
[]
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Thérèse
['https://www.wikidata.org/wiki/Q3527366']
Dieu
['https://www.wikidata.org/wiki/Q3027560']
Manot
['https://www.wikidata.org/wiki/Q2777738']
Clarisse
['https://www.wikidata.org/wiki/Q733658']
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
malin
['https://www.wikidata.org/wiki/Q1825744']
Manot
['https://www.wikidata.org/wiki/Q2777738']
Clarisse
['https://www.wikidata.org/wiki/Q733658']
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Tâche
[]
Manot
['https://www.wikida

['https://www.wikidata.org/wiki/Q3164791']
T'arrêteras-tu ? 

['https://www.wikidata.org/wiki/Q180309', 'https://www.wikidata.org/wiki/Q22909499']
Ah
['https://www.wikidata.org/wiki/Q869406']
Adonc
[]
Attrape
[]
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Roux
['https://www.wikidata.org/wiki/Q201157']
Tiens
[]
Assez
[]
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
lune
[]
Voilà
['https://www.wikidata.org/wiki/Q18395683']
Roux
['https://www.wikidata.org/wiki/Q201157']
Parlons
[]
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Saint-Brunelle
[]
Accepté
[]
Roux
['https://www.wikidata.org/wiki/Q201157']
Ma foi !
['https://www.wikidata.org/wiki/Q5464146']
Fallait
[]
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Dizy
['https://www.wikidata.org/wiki/Q741536']
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Souhaite
[]
tes griffes
[]
Clarisse Manot
['https://www.wikidata.org/wiki/Q733658', 'https:

['https://www.wikidata.org/wiki/Q22661961']
Assis
['https://www.wikidata.org/wiki/Q742227']
Ah !
['https://www.wikidata.org/wiki/Q869406']
Sortez
[]
la France
['https://www.wikidata.org/wiki/Q142']
la France
['https://www.wikidata.org/wiki/Q142']
Ah !
['https://www.wikidata.org/wiki/Q869406']
Gens de Morlinval
['https://www.wikidata.org/wiki/Q5533554']
Morlincourt
['https://www.wikidata.org/wiki/Q1336814']
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
la Révolution
['https://www.wikidata.org/wiki/Q3527389']
la France
['https://www.wikidata.org/wiki/Q142']
ça
[]
la France
['https://www.wikidata.org/wiki/Q142']
Argent !
['https://www.wikidata.org/wiki/Q2301367']
la France
['https://www.wikidata.org/wiki/Q142']
Reviendront
[]
Pourvu
[]
Céran
['https://www.wikidata.org/wiki/Q859610']
père Roux
['https://www.wikidata.org/wiki/Q201157']
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Cadet Lambin
['https://www.wikidata.org/wiki/Q61668028', 'https://www.wikidata.org/wiki/Q5624

...

['https://www.wikidata.org/wiki/Q682']
F
['https://www.wikidata.org/wiki/Q650']
.3
[]
Ag
['https://www.wikidata.org/wiki/Q1090']
L¬.
[]
Se & e   i

e eEt

['https://www.wikidata.org/wiki/Q82435']
-E-e
['https://www.wikidata.org/wiki/Q180309', 'https://www.wikidata.org/wiki/Q82435']
E
['https://www.wikidata.org/wiki/Q2091584']
ae-
[]
C 2
['https://www.wikidata.org/wiki/Q623']
ee
['https://www.wikidata.org/wiki/Q191']
AL
['https://www.wikidata.org/wiki/Q30449']
S rpe
['https://www.wikidata.org/wiki/Q682']
Mi
['https://www.wikidata.org/wiki/Q926323']
LeV
[]
S
['https://www.wikidata.org/wiki/Q682']
E E
['https://www.wikidata.org/wiki/Q2091584', 'https://www.wikidata.org/wiki/Q2091584']
.as¬
[]
TSriteo
[]
E   rDbee!
['https://www.wikidata.org/wiki/Q2091584']
e¬.
[]
Ceeeeee
[]
E
['https://www.wikidata.org/wiki/Q2091584']
A2
['https://www.wikidata.org/wiki/Q888603']
S-
[]
I
['https://www.wikidata.org/wiki/Q1103']
S e-E
 ¬
S.2
-e
¬
['https://www.wikidata.org/wiki/Q682', 'https://www.wikidata.o

['https://www.wikidata.org/wiki/Q16']
Clarisse
['https://www.wikidata.org/wiki/Q733658']
H
['https://www.wikidata.org/wiki/Q556']
Gellier
[]
MION VIILI.AGE
['https://www.wikidata.org/wiki/Q16043593']
Clarisse
['https://www.wikidata.org/wiki/Q733658']
-W
[]
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Ca
['https://www.wikidata.org/wiki/Q706']
Roux
['https://www.wikidata.org/wiki/Q201157']
prise d'Alger
['https://www.wikidata.org/wiki/Q3561']
MON
['https://www.wikidata.org/wiki/Q235']
V1LLAGE
[]
bahl
[]
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Jc
['https://www.wikidata.org/wiki/Q370258']
ete poltron
[]
Sylvain
['https://www.wikidata.org/wiki/Q16281827']
Nom-de-Nom
[]
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Jeacquet
[]
Amen
['https://www.wikidata.org/wiki/Q58373']
Vive le roi
['https://www.wikidata.org/wiki/Q61037170']
Manot
['https://www.wikidata.org/wiki/Q2777738']
Jeacquet
[]
hAfaot
[]
l--
[]
II1
[]
VILLAGE
['https://www.wikidata.org/wiki/Q465227'

[]
bon Dieu
['https://www.wikidata.org/wiki/Q3027560']
plut0
[]
Toinon
[]
sontMON
[]
Morlincourt
['https://www.wikidata.org/wiki/Q1336814']
bonte
[]
A1
['https://www.wikidata.org/wiki/Q266066']
Notre maire
['https://www.wikidata.org/wiki/Q382617']
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
niere
[]
IMION VILLAGE
[]
C0NSCRIPTION-
[]
CONSCRIPTION
['https://www.wikidata.org/wiki/Q5962411']
Pensez-y. Adonc
['https://www.wikidata.org/wiki/Q180309', 'https://www.wikidata.org/wiki/Q4021891']
sou
['https://www.wikidata.org/wiki/Q56508']
sou
['https://www.wikidata.org/wiki/Q56508']
Saint-Brunelle
[]
VILLAGE
['https://www.wikidata.org/wiki/Q465227']
renceP Pourront
[]
Craintifs
[]
Ml
['https://www.wikidata.org/wiki/Q2539']
Ceran
[]
deMlorlinval
[]
Ceran
[]
M0ON
[]
Fallut
[]
Pres de la fontaine Sainte-Radegonde
['https://www.wikidata.org/wiki/Q7234232', 'https://www.wikidata.org/wiki/Q838235', 'https://www.wikidata.org/wiki/Q581707']
Maria
['https://www.wikidata.org/wiki/Q2845']
V1LLA

[]
Baptiste
['https://www.wikidata.org/wiki/Q63538437']
Baptiste
['https://www.wikidata.org/wiki/Q63538437']
Tour
['https://www.wikidata.org/wiki/Q117934']
annces.8
M0N VILLAGE
['https://www.wikidata.org/wiki/Q465227']
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
ca
['https://www.wikidata.org/wiki/Q16']
Taisez
[]
pre Roux
['https://www.wikidata.org/wiki/Q201157']
IBrugnon
[]
M0N YILLAG
[]
ca
['https://www.wikidata.org/wiki/Q16']
Pater
['https://www.wikidata.org/wiki/Q731533']
Ave
['https://www.wikidata.org/wiki/Q2319007']
Norine
[]
Roseet
[]
soitr-----
[]
M0N
[]
madame la mairesse
['https://www.wikidata.org/wiki/Q17146837', 'https://www.wikidata.org/wiki/Q56539707']
Rose
['https://www.wikidata.org/wiki/Q519034']
na pro-
tection
[]
Rose
['https://www.wikidata.org/wiki/Q519034']
Inoi
['https://www.wikidata.org/wiki/Q30587709']
Cellicr
[]
mircsseMION VILLAGE
['https://www.wikidata.org/wiki/Q465227']
Rosc
[]
lPendant
[]
Woyons
[]
Norine
[]
pauvIe
[]
Pardonnez-lui
['https://www.wi

[]
Roux
['https://www.wikidata.org/wiki/Q201157']
renie-Dieu
['https://www.wikidata.org/wiki/Q511777', 'https://www.wikidata.org/wiki/Q3027560']
Honnete
[]
honnete
[]
enterrement198
[]
VILLAGE
['https://www.wikidata.org/wiki/Q465227']
soyez tranquilles
[]
ca
['https://www.wikidata.org/wiki/Q16']
Venus
['https://www.wikidata.org/wiki/Q313']
Berenice
['https://www.wikidata.org/wiki/Q196653']
Cassiopeel
[]
La Rose
['https://www.wikidata.org/wiki/Q519034']
ca
['https://www.wikidata.org/wiki/Q16']
Rose
['https://www.wikidata.org/wiki/Q519034']
soyez-en
[]
Madame
['https://www.wikidata.org/wiki/Q17146837']
VILLAGE
['https://www.wikidata.org/wiki/Q465227']
On
dirait que les familles
[]
Expliquez
[]
Chanvriers
[]
chnevis
[]
VILLAGE
['https://www.wikidata.org/wiki/Q465227']
II
L.
[]
Action
['https://www.wikidata.org/wiki/Q2634111']
a---------------a-------a--------
[]
Rose
['https://www.wikidata.org/wiki/Q519034']
madame la mairesse
['https://www.wikidata.org/wiki/Q17146837', 'https://www.wikid

## Nous comparons ensuite les URLs obtenues lors du linking pour les entités en contexte et hors contexte

In [11]:
import json
import spacy  # version 3.5
import glob
import pandas as pd

def lire_fichier(chemin):
    with open(chemin, "r", encoding="utf-8") as fichier:
        texte = fichier.read()
        return texte
    
def tableau(data):
    # Création du tableau récapitulatif
    df = pd.DataFrame(data)
    print (df)
    table_latex = df.to_latex(index=False)
    print(table_latex)


modele=["sm","md","lg"]
path_corpora = "../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/*/*.txt"
donnee = []

for mod in modele:
    print("Pour le modèle:", mod, "\n")
    nlp = spacy.load("fr_core_news_" + mod)
    nlp.add_pipe("entityfishing")
    for path in glob.glob(path_corpora):
        print("Entrée:", path)
        texte = lire_fichier(path)
        doc = nlp(texte)
        compteur = 0
        intersect = 0
        nb_entite = 0

        for ent in doc.ents:
            contexte = str(texte[ent.start_char-50:ent.start_char])+str(ent.text)+ str(texte[ent.end_char:ent.end_char+50])
            entite = ent.text
            
            liste_url_entourage = []
            liste_url_entite = []
            doc3 = nlp(entite)
            doc2 = nlp(contexte)
            for ent in doc2._.linkedEntities:
                url_entourage = ent.get_url()
                liste_url_entourage.append(url_entourage)
            for ent in doc3._.linkedEntities:
                url_entite = ent.get_url()
                liste_url_entite.append(url_entite)
                for d in liste_url_entite:
                    if d in liste_url_entourage:
                        intersect += 1
                    compteur += 1
            nb_entite += 1
        
        non = compteur - intersect
        print("La REN de Spacy a identifié", nb_entite, "entités")
        print("L'entity linker a repéré", compteur, "entités")
        print("Il y a", intersect, "éléments communs")
        print("Il y a", non, "éléments qui ne sont pas en commun")

        donnee.append({
            "Modèle": mod,
            "Fichier": path,
            "Entités reconnues par Spacy": nb_entite,
            "Entités reconnues par l'Entity Linker": compteur,
            "Entités avec URLs communes ": intersect,
            "Entités avec URLs non communes": non
        })
tableau(donnee)

Pour le modèle: md 

Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/REF/ADAM_Mon-village_REF.txt
La REN de Spacy a identifié 790 entités
L'entity linker a repéré 653 entités
Il y a 602 éléments communs
Il y a 51 éléments qui ne sont pas en commun
Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/OCR/ADAM_Mon-village_Kraken-base.txt
La REN de Spacy a identifié 1256 entités
L'entity linker a repéré 1072 entités
Il y a 817 éléments communs
Il y a 255 éléments qui ne sont pas en commun
  Modèle                                            Fichier  \
0     md  ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/RE...   
1     md  ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/OC...   

   Entités reconnues par Spacy  Entités reconnues par l'Entity Linker  \
0                          790                                    653   
1                         1256                                   1072   

   Entités avec URLs communes   Entités avec URLs non communes  
0                          602 

/tmp/ipykernel_3388/539984079.py:15: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  table_latex = df.to_latex(index=False)


## Nous avons étudié le nombre d'URL associé à chaque entité

In [12]:
import spacy 
import glob
import pandas as pd

def lire_fichier(chemin):
    with open(chemin, "r", encoding="utf-8") as fichier:
        texte = fichier.read()
        return texte
def tableau(data):
    # Création du tableau récapitulatif
    df = pd.DataFrame(data)
    print (df)
    table_latex = df.to_latex(index=False)
    print(table_latex)

modele=["sm","md","lg"]

path_corpora = "../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/*/*.txt"
data = []
tab_entite = []
for mod in modele:
    print("Pour le modèle:", mod, "\n")
    nlp = spacy.load("fr_core_news_" + mod)
    nlp.add_pipe("entityLinker", last=True)
    for path in glob.glob(path_corpora):
        print("Entrée:", path)
        texte = lire_fichier(path)
        doc = nlp(texte)
        compteur = 0
        intersect = 0
        nb_entite = 0

        for ent in doc.ents:
            contexte = str(texte[ent.start_char-50:ent.start_char])+str(ent.text)+ str(texte[ent.end_char:ent.end_char+50])
            entite = ent.text
            #print ("pour l'entité", entite)
            liste_url_entourage = []
            liste_url_entite = []
            doc3 = nlp(entite)
            doc2 = nlp(contexte)
            for ent in doc2._.linkedEntities:
                url_entourage = ent.get_url()
                liste_url_entourage.append(url_entourage)
            for ent in doc3._.linkedEntities:
                url_entite = ent.get_url()
                liste_url_entite.append(url_entite)
                #print("Il y a",len(liste_url_entite),"url correspondantes. Ce sont", liste_url_entite)
                tab_entite.append({
                    "Modèle": mod,
                    "Fichier": path,
                    "Entité reconnue par la REN de SpaCy": entite,
                    "nb d'entités repérées par l'Entity Linker pour cette entité": len(liste_url_entite),
                    "url(s) liée(s) par l'Entity Linker pour cette entité": liste_url_entite,
                })

            for d in liste_url_entite:
                if d in liste_url_entourage:
                    intersect += 1
                compteur += 1
            nb_entite += 1
        
        non = compteur - intersect
        #print("La REN de Spacy a identifié", nb_entite, "entités")
        #print("L'entity linker a repéré", compteur, "entités")
        #print("Il y a", intersect, "éléments communs")
        #print("Il y a", non, "éléments qui ne sont pas en communs")
        
        data.append({
            "Modèle": mod,
            "Fichier": path,
            "Entités Spacy": nb_entite,
            "Entités Entity Linker": compteur,
            "Entités communes": intersect,
            "Entités non communes": non
        })
        
        
tableau(tab_entite)


Pour le modèle: sm 

Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/REF/ADAM_Mon-village_REF.txt
Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/OCR/ADAM_Mon-village_Kraken-base.txt
Pour le modèle: md 

Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/REF/ADAM_Mon-village_REF.txt
Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/OCR/ADAM_Mon-village_Kraken-base.txt
Pour le modèle: lg 

Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/REF/ADAM_Mon-village_REF.txt
Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/OCR/ADAM_Mon-village_Kraken-base.txt
     Modèle                                            Fichier  \
0        sm  ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/RE...   
1        sm  ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/RE...   
2        sm  ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/RE...   
3        sm  ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/RE...   
4        sm  ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/RE...   
...     ...                    

/tmp/ipykernel_3388/2682745239.py:13: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  table_latex = df.to_latex(index=False)


\begin{tabular}{lllrl}
\toprule
Modèle &                                            Fichier &                Entité reconnue par la REN de SpaCy &  nb d'entités repérées par l'Entity Linker pour cette entité & url(s) liée(s) par l'Entity Linker pour cette entité \\
\midrule
    sm & ../DATA/ELTeC-Fra\_2023/ADAM/ADAM\_krakenbase/REF... &                                      Norine Duclos &                                                  1 &          [https://www.wikidata.org/wiki/Q21450529] \\
    sm & ../DATA/ELTeC-Fra\_2023/ADAM/ADAM\_krakenbase/REF... &                                        Jean-Claude &                                                  1 &           [https://www.wikidata.org/wiki/Q3164791] \\
    sm & ../DATA/ELTeC-Fra\_2023/ADAM/ADAM\_krakenbase/REF... &                                               Rose &                                                  1 &            [https://www.wikidata.org/wiki/Q519034] \\
    sm & ../DATA/ELTeC-Fra\_2023/ADAM/ADAM\_krakenbas

## Entités qui n'ont pas les mêmes URLs liées en contexte et hors contexte

L'influence du contexte semble assez marginale. En effet, immuablement "Pierre" est toujours associé à la capitale du Dakota du Sud quel que soit le contexte. L'écart entre les entités reconnues en contexte et hors contexte ne semble pas lié au linking lui-même mais plutôt aux écart de performance entre la REN de Spacy et celle de l'EL. En effet, dans une même phrase, les entités reconnues comme étant des EN ne sont pas les mêmes. 

Par exemple " Clarisse" est identifiée comme une EN par Spacy. L'entity Linker hors contexte va l'associer à  "Poor Clares: Catholic order of convent nuns" et en contexte il ne va même pas reconnaitre l'entité. 
Ainsi pour une même entité, on ne trouve pas d'écart de définition. 

In [5]:
import spacy
import os
import json


modele=["fr_core_news_sm","fr_core_news_md","fr_core_news_lg"]

path_corpora = "../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/OCR/*.txt"
#path_corpora = "../DATA/ELTeC-Fra_2023/*/*/*/*.txt"


#modele = ["fr_core_news_sm"]

bilan = []
tab_entite = []
tab_noncommun = []
tab_basename = []

for mod in modele:
    print("Pour le modèle:", mod, "\n")
    nlp = spacy.load(mod)
    nlp.add_pipe("entityLinker", last=True)
    nb_ent_NER= 0
    nb_ent_EL = 0
    nb_entite_tt = 0
    for path in glob.glob(path_corpora):
        basename = os.path.basename(path)
        print("Entrée:", path)
        texte = lire_fichier(path)
        
        doc = nlp(texte)
        compteur = 0
        intersect = 0
        nb_entite = 0

        for ent in doc.ents:
            nb_entite_tt+=1
            contexte = str(texte[ent.start_char-200:ent.start_char]) + str(ent.text) + str(texte[ent.end_char:ent.end_char+200])
            entite = ent.text
            
            liste_url_entourage = []
            liste_url_entite = []
            liste_label_entourage = []
            liste_label_entite = []
            liste_description_entourage = []
            liste_description_entite = []
            liste_concat_entourage = []
            liste_concat_entite = []
            
            doc3 = nlp(entite)
            doc2 = nlp(contexte)
            
            for ent in doc2._.linkedEntities:
                url_entourage = ent.get_url()
                label_entourage = ent.get_label()
                description_entourage = ent.get_description()
                liste_url_entourage.append(url_entourage)
                liste_label_entourage.append(label_entourage)
                liste_description_entourage.append(description_entourage)
                concat_entourage = f"{label_entourage}: {description_entourage}"
                liste_concat_entourage.append(concat_entourage)
            
            for ent in doc3._.linkedEntities:
                url_entite = ent.get_url()
                nom_entite = ent.get_label()
                description_entite = ent.get_description()
                concat_entite = f"{nom_entite}: {description_entite}"
                liste_url_entite.append(url_entite)
                liste_concat_entite.append(concat_entite)
                
                tab_entite.append({
                    "Modèle": mod,
                    "Fichier": basename,
                    "Entité reconnue par la REN de SpaCy": entite,
                    "nb d'entités repérées par l'Entity Linker": len(liste_url_entite),
                    "Entités reconnues par l'Entity Linker": liste_concat_entite,
                    "urls des entités reconnues par l'Entity Linker": liste_url_entite
                })

            for d in liste_url_entite:
                compteur += 1
                if d in liste_url_entourage:
                    intersect += 1
                
                else:
                    tab_noncommun.append({
                        "Modèle": mod,
                        "Fichier": basename,
                        "Entité reconnue par la REN de SpaCy": entite,
                        "contexte d'apparition de l'entité" : contexte,
                        "entités reconnues pour l'Entity Linker sans contexte": liste_concat_entite,
                        "url des entités reconnues pour l'Entity Linker sans contexte": liste_url_entite,
                        "entités reconnues pour l'Entity Linker en contexte": liste_concat_entourage,
                        "url des entités reconnues pour l'Entity Linker en contexte": liste_url_entourage
                    })
                    
            nb_entite += 1
        
        non = compteur - intersect
        
        tableau(tab_entite, path,"url_par_entite")
        
        tableau(tab_noncommun, path,"urls_non_communes")
        bilan.append({
            "Modèle": mod,
            "Fichier": basename,
            "Entités Spacy": nb_entite,
            "Entités Entity Linker": compteur,
            "Entités communes": intersect,
            "Entités non communes": non
        })
#tableau(bilan, path, "bilan")


t1 = time.time()
total = t1 - t0
print (non)
print("Le code a mis", total, "secondes pour s'exécuter")


Pour le modèle: fr_core_news_sm 

Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/OCR/ADAM_Mon-village_Kraken-base.txt


NameError: name 'tableau' is not defined